In [122]:
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt

from sktime.utils.load_data import load_from_tsfile_to_dataframe

from gtda.time_series import TakensEmbedding
from gtda.homology import VietorisRipsPersistence
from gtda.diagrams import Scaler, Filtering

from gtda.plotting import plot_diagram

from abc import ABC

## 1. Common routines for extracting Topological Features

In [4]:
"""
Abstract class for creating Topology Features extractors.
"""
class TDAFeatureExtractor(ABC):
    def extract(X):
        pass

In [135]:
class PersistenceDiagramsExtractor:
    def __init__(self, takens_embedding_dim, takens_embedding_delay, homology_dimensions, 
                 filtering=False, filtering_dimensions=(1, 2)):
        self.takens_embedding_dim_ = takens_embedding_dim
        self.takens_embedding_delay_ = takens_embedding_delay
        self.homology_dimensions_ = homology_dimensions
        self.filtering_ = filtering
        self.filtering_dimensions_ = filtering_dimensions
        
    def takens_embeddings_(self, X):
        X_transformed = list()
        for series in X:
            te = TakensEmbedding(parameters_type='search', 
                                 dimension=self.takens_embedding_dim_, 
                                 time_delay=self.takens_embedding_delay_)
            X_transformed.append(te.fit_transform(series))
        return X_transformed
    
    def persistence_diagrams_(self, X_embdeddings):
        X_transformed = list()
        for embedding in X_embdeddings:
            vr = VietorisRipsPersistence(metric='euclidean', 
                                         max_edge_length=100, 
                                         homology_dimensions=self.homology_dimensions_)
            diagram_scaler = Scaler()
            persistence_diagrams = diagram_scaler.fit_transform(vr.fit_transform([embedding]))
            if self.filtering_:
                diagram_filter = Filtering(epsilon=0.1, homology_dimensions=self.filtering_dimensions_)
                persistence_diagrams = diagram_filter.fit_transform(persistence_diagrams)
            X_transformed.append(persistence_diagrams[0])
        return X_transformed
        
    def fit_transform(self, X):
        X_embeddings = self.takens_embeddings_(X)
        X_persistence_diagrams = self.persistence_diagrams_(X_embeddings)
        return X_persistence_diagrams
        

## 2. Example with Time Series

In [107]:
def pd_to_np(df):
    X_np = list()
    for row in df["dim_0"]:
        X_np.append(row.to_numpy())
    return np.array(X_np)

def load_dataset(dataset_name):
    X_train_pd, y_train_pd = load_from_tsfile_to_dataframe("Univariate_ts/" + dataset_name + "/" + dataset_name + "_TRAIN.ts") 
    X_test_pd, y_test_pd = load_from_tsfile_to_dataframe("Univariate_ts/" + dataset_name + "/" + dataset_name + "_TEST.ts")
    return pd_to_np(X_train_pd), y_train_pd.astype(int), pd_to_np(X_test_pd), y_test_pd.astype(int)

In [113]:
X_train, y_train, X_test, y_test = load_dataset("MoteStrain")

In [144]:
extractor = PersistenceDiagramsExtractor(takens_embedding_dim=3,
                                         takens_embedding_delay=10,
                                         homology_dimensions=(0, 1, 2),
                                         filtering=False)
diagrams = extractor.fit_transform(X_train)

In [145]:
plot_diagram(diagrams[2])